In [1]:
import pandas as pd
import numpy as np

import json

In [43]:
fire = pd.read_csv('assets/data/Fire_Department_Calls_for_Service_031020_032520.csv')
print(fire.shape)
fire.head()

(13202, 34)


,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,...,ALS Unit,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,Location,RowID
0,200760779,E14,20033071,Medical Incident,03/16/2020,03/16/2020,03/16/2020 08:46:41 AM,03/16/2020 08:48:42 AM,03/16/2020 08:49:47 AM,03/16/2020 08:50:54 AM,...,True,Potentially Life-Threatening,1,ENGINE,1,7,1,Outer Richmond,"(37.773108263051476, -122.49290116792024)",200760779-E14
1,200763117,59,20033351,Medical Incident,03/16/2020,03/16/2020,03/16/2020 08:02:25 PM,03/16/2020 08:04:02 PM,03/16/2020 08:04:20 PM,03/16/2020 08:04:28 PM,...,True,Potentially Life-Threatening,1,MEDIC,3,9,11,Excelsior,"(37.72512194367752, -122.43350454859788)",200763117-59
2,200723829,E14,20031679,Medical Incident,03/12/2020,03/12/2020,03/12/2020 09:07:57 PM,03/12/2020 09:08:43 PM,03/12/2020 09:09:25 PM,03/12/2020 09:10:27 PM,...,True,Potentially Life-Threatening,1,ENGINE,1,7,1,Outer Richmond,"(37.78224119276634, -122.48112917551705)",200723829-E14
3,200760153,KM03,20033001,Medical Incident,03/16/2020,03/15/2020,03/16/2020 01:57:18 AM,03/16/2020 01:59:11 AM,03/16/2020 01:59:47 AM,03/16/2020 02:01:19 AM,...,False,Potentially Life-Threatening,1,PRIVATE,2,2,6,Tenderloin,"(37.78326038612958, -122.4183661541803)",200760153-KM03
4,200714268,T06,20031263,Other,03/11/2020,03/11/2020,03/11/2020 08:48:25 PM,03/11/2020 08:48:25 PM,03/11/2020 08:49:04 PM,03/11/2020 08:50:31 PM,...,False,Alarm,1,TRUCK,1,5,8,Castro/Upper Market,"(37.76819748043482, -122.4356401929219)",200714268-T06


In [44]:
# check column info and dtypes
fire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13202 entries, 0 to 13201
Data columns (total 34 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Call Number                           13202 non-null  int64  
 1   Unit ID                               13202 non-null  object 
 2   Incident Number                       13202 non-null  int64  
 3   Call Type                             13202 non-null  object 
 4   Call Date                             13202 non-null  object 
 5   Watch Date                            13202 non-null  object 
 6   Received DtTm                         13202 non-null  object 
 7   Entry DtTm                            13202 non-null  object 
 8   Dispatch DtTm                         13202 non-null  object 
 9   Response DtTm                         12941 non-null  object 
 10  On Scene DtTm                         10745 non-null  object 
 11  Transport DtTm 

In [45]:
# convert columns to proper dtypes
fire = fire.convert_dtypes()
fire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13202 entries, 0 to 13201
Data columns (total 34 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Call Number                           13202 non-null  Int64  
 1   Unit ID                               13202 non-null  string 
 2   Incident Number                       13202 non-null  Int64  
 3   Call Type                             13202 non-null  string 
 4   Call Date                             13202 non-null  string 
 5   Watch Date                            13202 non-null  string 
 6   Received DtTm                         13202 non-null  string 
 7   Entry DtTm                            13202 non-null  string 
 8   Dispatch DtTm                         13202 non-null  string 
 9   Response DtTm                         12941 non-null  string 
 10  On Scene DtTm                         10745 non-null  string 
 11  Transport DtTm 

In [46]:
# convert call date column to datetime
fire['Call Date'] = pd.to_datetime(fire['Call Date'])

In [47]:
# create column to indicate whether the call (i.e. row) belongs to the pre-lockdown or lockdown category 
# pre-lockdown < 03/16/2020, lockdown >= 03/16/2020 
fire['Lockdown'] = 'Pre-Lockdown'
fire.loc[fire['Call Date']>='2020-03-16', 'Lockdown'] = 'Lockdown'
fire.head()

,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,...,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,Location,RowID,Lockdown
0,200760779,E14,20033071,Medical Incident,2020-03-16,03/16/2020,03/16/2020 08:46:41 AM,03/16/2020 08:48:42 AM,03/16/2020 08:49:47 AM,03/16/2020 08:50:54 AM,...,Potentially Life-Threatening,1,ENGINE,1,7,1,Outer Richmond,"(37.773108263051476, -122.49290116792024)",200760779-E14,Lockdown
1,200763117,59,20033351,Medical Incident,2020-03-16,03/16/2020,03/16/2020 08:02:25 PM,03/16/2020 08:04:02 PM,03/16/2020 08:04:20 PM,03/16/2020 08:04:28 PM,...,Potentially Life-Threatening,1,MEDIC,3,9,11,Excelsior,"(37.72512194367752, -122.43350454859788)",200763117-59,Lockdown
2,200723829,E14,20031679,Medical Incident,2020-03-12,03/12/2020,03/12/2020 09:07:57 PM,03/12/2020 09:08:43 PM,03/12/2020 09:09:25 PM,03/12/2020 09:10:27 PM,...,Potentially Life-Threatening,1,ENGINE,1,7,1,Outer Richmond,"(37.78224119276634, -122.48112917551705)",200723829-E14,Pre-Lockdown
3,200760153,KM03,20033001,Medical Incident,2020-03-16,03/15/2020,03/16/2020 01:57:18 AM,03/16/2020 01:59:11 AM,03/16/2020 01:59:47 AM,03/16/2020 02:01:19 AM,...,Potentially Life-Threatening,1,PRIVATE,2,2,6,Tenderloin,"(37.78326038612958, -122.4183661541803)",200760153-KM03,Lockdown
4,200714268,T06,20031263,Other,2020-03-11,03/11/2020,03/11/2020 08:48:25 PM,03/11/2020 08:48:25 PM,03/11/2020 08:49:04 PM,03/11/2020 08:50:31 PM,...,Alarm,1,TRUCK,1,5,8,Castro/Upper Market,"(37.76819748043482, -122.4356401929219)",200714268-T06,Pre-Lockdown


In [48]:
# select the columns for the visualization
viz_cols = ['City', 'Lockdown', 'Call Type Group', 'Call Type', 'Neighborhooods - Analysis Boundaries']
filtered = fire.loc[:, viz_cols]
filtered.head()

,City,Lockdown,Call Type Group,Call Type,Neighborhooods - Analysis Boundaries
0,San Francisco,Lockdown,Potentially Life-Threatening,Medical Incident,Outer Richmond
1,San Francisco,Lockdown,Potentially Life-Threatening,Medical Incident,Excelsior
2,San Francisco,Pre-Lockdown,Potentially Life-Threatening,Medical Incident,Outer Richmond
3,San Francisco,Lockdown,Potentially Life-Threatening,Medical Incident,Tenderloin
4,San Francisco,Pre-Lockdown,Alarm,Other,Castro/Upper Market


In [49]:
# check for any NAs 
print(filtered.isna().sum())

# remove NA rows
filtered = filtered.dropna()
filtered.shape

City                                      0
Lockdown                                  0
Call Type Group                         117
Call Type                                 0
Neighborhooods - Analysis Boundaries      0
dtype: int64


(13085, 5)

In [53]:
# rename Neighborhooods - Analysis Boundaries to make it easier to work with in javascript
filtered.rename(columns={'Neighborhooods - Analysis Boundaries': 'Neighborhood'}, inplace=True)

In [54]:
# write data to csv
filtered.to_csv('assets/data/filtered_fire_calls.csv', index=False)

In [52]:
filtered['Call Type Group'].unique()

<StringArray>
['Potentially Life-Threatening', 'Alarm', 'Non Life-threatening', 'Fire']
Length: 4, dtype: string